# Proyecto Final: Clasificación de tópicos de interés

## Exploración de datos


CC5113 - Aprendizaje Automático Bayesiano

Profesor: Pablo Guerrero

Autor: Martín Cornejo

## Paquetes necesarios

In [66]:
import numpy as np
import matplotlib.pyplot as plt
import pdb
import itertools
import operator

import pandas as pd
from nltk.stem.snowball import SpanishStemmer

In [43]:
#data = np.genfromtxt('data_format.csv', delimiter='')
datos=pd.read_csv('data_format.csv')
print(datos.shape)
#pdb.set_trace()
datos.head()

(138, 2)


,Texto,Interes
0,"Ojalá obliguen a Piñera a cerrar Punta Peuco, ...",False
1,Piñera para crear base de apoyo moderada a su ...,True
2,@CNNChile MEMORIA 2014 Adimark: Piñera termina...,True
3,PPK y Piñera en privado habrían conversado alg...,False
4,Bachelet entregará el gobierno de Chile a Piñera,True


## Limpiando strings

### Reemplazar tildes, caracteres especiales, todo a minúsculas

In [42]:
strings = datos.ix[:,0]
tildes = ['á','é','í','ó','ú']
vocales = ['a','e','i','o','u']

# tildes
for idx, vocal in enumerate(vocales):
    strings = strings.str.replace(tildes[idx],vocal)

# caracteres especiales menos la ñ
strings = strings.str.replace('[^a-zñA-Z ]', "")

# todo a minusculas
strings = pd.Series(list(map(lambda x: x.lower(), strings)))
print(strings.shape)
strings.head()

(138,)


0    ojala obliguen a piñera a cerrar punta peuco e...
1    piñera para crear base de apoyo moderada a su ...
2    cnnchile memoria  adimark piñera termina su go...
3    ppk y piñera en privado habrian conversado alg...
4     bachelet entregara el gobierno de chile a piñera
dtype: object

## Stemming

In [87]:
stemmer = SpanishStemmer()

# oracion a arreglo de strings
strings_arr = list(map(lambda x: x.split(), strings))
strings_arr = list(itertools.chain.from_iterable(strings_arr))
root_arr = list(map(lambda x: stemmer.stem(x), strings_arr))

def most_common(L):
  # get an iterable of (item, iterable) pairs
  SL = sorted((x, i) for i, x in enumerate(L))
  list_pairs = []
  #print('SL:', SL)
  groups = itertools.groupby(SL, key=operator.itemgetter(0))
  # auxiliary function to get "quality" for an item
  def _auxfun(g):
    item, iterable = g
    count = 0
    min_index = len(L)
    for _, where in iterable:
      count += 1
      min_index = min(min_index, where)
    list_pairs.append((item, count))
    #print('item %r, count %r, minind %r' % (item, count, min_index))
    return count, -min_index
  # pick the highest-count/earliest item
  return max(groups, key=_auxfun)[0], list_pairs

most_common, list_pairs = most_common(root_arr)

pares_filtrados = list(filter(lambda x: len(x[0]) > 2, list_pairs))

common_roots_sorted = sorted(pares_filtrados, key=lambda tup: tup[1], reverse=True)
#common_roots_sorted